In [1]:

import rbdl
import numpy as np
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
import pybullet as pb

root = 'base_link'
tip = 'tilt_link'
path_to_urdf = '/home/lmjohann/urdf2casadi/examples/urdf/pantilt.urdf'
ok, tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = tree.getChain(root,tip)
rbdlmodel = rbdl.loadModel(path_to_urdf)


sim = pb.connect(pb.DIRECT)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
pb.setGravity(0, 0, -9.81)
q_max = [3.14, 4.64]
q_min = [-3.13, -4.64]
n_joints = 2

q_kdl = kdl.JntArray(n_joints)
tau_kdl = kdl.JntArray(n_joints)
gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81


q_pb = [None]*n_joints
zeros_pb = [None]*n_joints

#rbdl
q_rbdl = np.zeros(n_joints)
tau_rbdl = np.zeros(n_joints)
zeros_rbdl = np.zeros(n_joints)



error_kdl_rbdl = np.zeros(n_joints)
error_pb_rbdl = np.zeros(n_joints)
error_pb_kdl = np.zeros(n_joints)

def list2np(asd):
    return np.asarray(asd)

n_itr = 1000
for i in range(n_itr):
    for j in range(n_joints):
        q_pb[j] = (q_max[j] - q_min[j])*np.random.rand()-(q_max[j] - q_min[j])/2
        q_kdl[j] = q_pb[j]
        q_rbdl[j] = q_pb[j]
        zeros_pb[j] = 0.

    rbdl.InverseDynamics(rbdlmodel, q_rbdl, zeros_rbdl, zeros_rbdl, tau_rbdl)
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, tau_kdl)
    tau_pb = pb.calculateInverseDynamics(pbmodel, q_pb, zeros_pb, zeros_pb)


    for tau_idx in range(n_joints):
        error_kdl_rbdl[tau_idx] += np.absolute((list2np(tau_kdl[tau_idx]) - tau_rbdl[tau_idx]))
        error_pb_rbdl[tau_idx] += np.absolute((list2np(tau_pb[tau_idx]) - (tau_rbdl[tau_idx])))
        error_pb_kdl[tau_idx] += np.absolute((list2np(tau_kdl[tau_idx]) - list2np(tau_pb[tau_idx])))


sum_error_kdl_rbdl = 0
sum_error_pb_rbdl = 0
sum_error_pb_kdl = 0

for err in range(n_joints):
    sum_error_kdl_rbdl += error_kdl_rbdl[err]
    sum_error_pb_kdl += error_pb_kdl[err]
    sum_error_pb_rbdl += error_pb_rbdl[err]

print "\nSum of errors KDL vs. RBDL for", n_itr, "iterations:\n", sum_error_kdl_rbdl
print "\nSum of errors KDL vs. pybullet for", n_itr, "iterations:\n", sum_error_pb_kdl
print "\nSum of errors RBDL vs. pybullet for", n_itr, "iterations:\n",sum_error_pb_rbdl


The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.

Sum of errors KDL vs. RBDL for 1000 iterations:
2.243427665860054e-12

Sum of errors KDL vs. pybullet for 1000 iterations:
0.0014698499373702045

Sum of errors RBDL vs. pybullet for 1000 iterations:
0.0014698499360258355


In [2]:
#print "mass:", pb.getDynamicsInfo(pbmodel, 0)[0]
#print "friction:", pb.getDynamicsInfo(pbmodel, 0)[1]
#print "inertial vectior:", pb.getDynamicsInfo(pbmodel, 0)[2]
#print "local inertial pos:", pb.getDynamicsInfo(pbmodel, 0)[3]
#print "local inertial orn:", pb.getDynamicsInfo(pbmodel, 0)[4]
#print "restitution:", pb.getDynamicsInfo(pbmodel, 0)[5]
#print "rolling friction:", pb.getDynamicsInfo(pbmodel, 0)[6]
#print "spinning friction:", pb.getDynamicsInfo(pbmodel, 0)[7]
#print "damping:", pb.getDynamicsInfo(pbmodel, 0)[8]
#print "stifness:", pb.getDynamicsInfo(pbmodel, 0)[9]

In [3]:
#pb.changeDynamics(pbmodel, 0, contactStiffness = -1, contactDamping = -1 )
#pb.changeDynamics(pbmodel, 1, contactStiffness = -1, contactDamping = -1 )


In [4]:
n_itr = 1000
for i in range(n_itr):
    for j in range(n_joints):
        q[j] = (q_max[j] - q_min[j])*np.random.rand()-(q_max[j] - q_min[j])/2
        q_kdl[j] = q[j]
        q_np[j] = q[j]
        zeros_pb[j] = 0.

    rbdl.InverseDynamics(urmodel, q_np, zeros_rbdl, zeros_rbdl, g_rbdl)
    kdl.ChainDynParam(ur_chain, gravity_kdl).JntToGravity(q_kdl, g_kdl)
    g_pb = pb.calculateInverseDynamics(pbmodel, q, zeros_pb, zeros_pb)
    g_u2c = g_sym(q)
    #print g_u2c

    for tau_idx in range(n_joints):
        g_error_kdl_rbdl[tau_idx] += np.absolute((list2np(g_kdl[tau_idx]) - g_rbdl[tau_idx]))
        g_error_kdl_u2c[tau_idx] += np.absolute((list2np(g_kdl[tau_idx]) - u2c2np(g_u2c[tau_idx])))
        g_error_rbdl_u2c[tau_idx] += np.absolute((u2c2np(g_u2c[tau_idx]) - g_rbdl[tau_idx]))
        g_error_pb_u2c[tau_idx] += np.absolute((u2c2np(g_u2c[tau_idx]) - list2np(g_pb[tau_idx])))


sum_error_kdl_rbdl = 0
sum_error_kdl_u2c = 0
sum_error_rbdl_u2c = 0
sum_error_pb_u2c = 0

for err in range(n_joints):
    sum_error_kdl_rbdl += g_error_kdl_rbdl[err]
    sum_error_kdl_u2c += g_error_kdl_u2c[err]
    sum_error_rbdl_u2c += g_error_rbdl_u2c[err]
    sum_error_pb_u2c += g_error_pb_u2c[err]

print "\nSum of errors KDL vs. RBDL for", n_itr, "iterations:\n", sum_error_kdl_rbdl
print "\nSum of errors KDL vs. U2C for", n_itr, "iterations:\n", sum_error_kdl_u2c
print "\nSum of errors RBDL vs. U2C for", n_itr, "iterations:\n",sum_error_rbdl_u2c
print "\nSum of errors pybullet vs. U2C for", n_itr, "iterations:\n", sum_error_pb_u2c

NameError: name 'q' is not defined